In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from frvrs_utils import FRVRSUtilities
from notebook_utils import NotebookUtilities
import os
import os.path as osp
import pandas as pd
import re

nu = NotebookUtilities(
    data_folder_path=osp.abspath('../data'),
    saves_folder_path=osp.abspath('../saves')
)
fu = FRVRSUtilities(
    data_folder_path=osp.abspath('../data'),
    saves_folder_path=osp.abspath('../saves')
)


# Build and deidentify the dataset of OSU Simulation Session Logs

In [3]:

# Get whether we can bring the dataset up from storage
pickle_loaded = False
if nu.pickle_exists('frvrs_logs_df'):
    try:
        frvrs_logs_df = nu.load_object('frvrs_logs_df')
        pickle_loaded = True
    except AttributeError as e: print(str(e).strip())
if (not pickle_loaded) and nu.csv_exists('frvrs_logs_df', folder_path=nu.saves_folder):
    frvrs_logs_df = nu.load_csv(csv_name='frvrs_logs_df', folder_path=nu.saves_folder)
    pickle_loaded = True
pickle_loaded

True

In [4]:

# Get all logs into one data frame
if not pickle_loaded:
    
    # Add the CSVs to the data frame
    frvrs_logs_df = fu.concatonate_logs()
    
    # Remove numerically-named columns
    columns_list = [x for x in frvrs_logs_df.columns if not re.search(r'\d+', str(x))]
    frvrs_logs_df = frvrs_logs_df[columns_list]
    
    # Convert 'TRUE' and 'FALSE' to boolean values
    for cn in [
        'injury_record_injury_treated_with_wrong_treatment', 'injury_record_injury_treated',
        'injury_treated_injury_treated_with_wrong_treatment', 'injury_treated_injury_treated'
    ]: frvrs_logs_df[cn] = frvrs_logs_df[cn].map({'TRUE': True, 'FALSE': False, 'True': True, 'False': False})
    
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
print(frvrs_logs_df.shape) # (829116, 106)
columns_list = [cn for cn in frvrs_logs_df.columns if 'appl' in cn]
mask_series = False
for cn in columns_list: mask_series |= ~frvrs_logs_df[cn].isnull()
df = frvrs_logs_df[mask_series][columns_list]
display(df.sample(min(4, df.shape[0])).dropna(axis='columns', how='all').T)

(829116, 114)


,628492,818799,582745,731487
tool_applied_type,Tourniquet,NaN,NaN,Gauze_Dressing
tool_applied_attachment_point,RightUpLeg (UnityEngine.GameObject),NaN,NaN,skinCollider_BodyCollideLOD (UnityEngine.GameO...
tool_applied_tool_location,tor_ring (1) (UnityEngine.GameObject),NaN,NaN,NaN
tool_applied_data,tourniquet(Clone) (UnityEngine.GameObject),NaN,NaN,NaN
tool_applied_sender,AppliedTourniquet,NaN,NaN,AppliedDressingGauze
tag_applied_patient_id,NaN,Marine with Leg Amputation Root,Gary_1 Root,NaN
tag_applied_type,NaN,red,black,NaN
tool_applied_patient_id,Mike_7 Root,NaN,NaN,Mike_2 Root



## Check for duplicate file ingestion

In [5]:

# Filter all the rows that have more than one unique value in the file_name column for each value in the session_uuid column
mask_series = (frvrs_logs_df.groupby('session_uuid').file_name.transform(pd.Series.nunique) > 1)
assert frvrs_logs_df[mask_series].shape[0] == 0, "You have duplicate files"
# columns_list = ['session_uuid', 'file_name']
# for (session_uuid, file_name), df in frvrs_logs_df[mask_series][columns_list].drop_duplicates().sort_values(columns_list).groupby(columns_list):
#     if not file_name.startswith('Double runs removed/'):
#         file_path = osp.join(fu.data_logs_folder, *file_name.split('/'))
#         os.remove(file_path)


## Add new features according to your increasing domain knowledge

In [6]:

# Modalize into one patient ID column if possible
new_column_name = 'patient_id'
if (new_column_name not in frvrs_logs_df.columns):
    columns_list= [
        'patient_demoted_id', 'patient_record_id', 'injury_record_patient_id', 's_a_l_t_walk_if_can_patient_id',
        's_a_l_t_walked_patient_id', 's_a_l_t_wave_if_can_patient_id', 's_a_l_t_waved_patient_id', 'patient_engaged_id',
        'pulse_taken_patient_id', 'injury_treated_patient_id', 'tool_applied_patient_id', 'tag_applied_patient_id',
        'player_gaze_patient_id'
    ]
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, columns_list, new_column_name)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 107)
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

,record_count
patient_id,
Asian Bob_4 Root,40
Bob_0 Root,4599
Bob_1 Root,85
Bob_10 Root,568
Bob_12 Root,106
Bob_4 Root,118
Bob_7 Root,98
Bob_9 Root,30678
Gary_0 Root,457


In [7]:

# Modalize into one location ID column if possible
new_column_name = 'location_id'
if (new_column_name not in frvrs_logs_df.columns):
    columns_list= [
        'teleport_location', 'patient_demoted_position', 'patient_record_position', 'injury_record_injury_injury_locator',
        's_a_l_t_walk_if_can_sort_location', 's_a_l_t_walked_sort_location', 's_a_l_t_wave_if_can_sort_location',
        's_a_l_t_waved_sort_location', 'patient_engaged_position', 'bag_access_location', 'injury_treated_injury_injury_locator',
        'bag_closed_location', 'tag_discarded_location', 'tool_discarded_location', 'player_location_location',
        'player_gaze_location'
    ]
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, columns_list, new_column_name)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 108)
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

,record_count
location_id,
"(-0.1, 0.0, -0.2)",3
"(-0.1, 0.0, -0.5)",2
"(-0.1, 0.0, -0.7)",1
"(-0.1, 0.0, -1.0)",1
"(-0.1, 0.0, -1.7)",2
...,...
"(8.7, 0.0, -3.1)",1
"(8.8, 0.0, -1.7)",1
"(8.9, 0.0, -1.6)",1


In [8]:

# Modalize into one injury ID column if possible
new_column_name = 'injury_id'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, ['injury_record_id', 'injury_treated_id'], new_column_name)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 109)
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

,record_count
injury_id,
Asthmatic,60
Ear Bleed,418
Face Shrapnel,816
Forehead Scrape,380
L Bicep Puncture,33
L Calf Laceration,54
L Calf Shrapnel,7
L Chest Collapse,364
L Forearm Laceration,801


In [9]:

# Modalize into one patient sort column if possible
new_column_name = 'patient_sort'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, ['patient_demoted_sort', 'patient_record_sort', 'patient_engaged_sort'], new_column_name)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 110)
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

,record_count
patient_sort,
still,5438
walker,4960
waver,8196


In [10]:

# Any runs longer than that 16 minutes are probably an instance
# of someone taking off the headset and setting it on the ground.
# 1 second = 1,000 milliseconds; 1 minute = 60 seconds
new_column_name = 'is_scene_aborted'
# if (new_column_name in frvrs_logs_df.columns): frvrs_logs_df = frvrs_logs_df.drop(columns=new_column_name)
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df[new_column_name] = False
    for (session_uuid, scene_id), scene_df in frvrs_logs_df.groupby(fu.scene_groupby_columns):
        mask_series = True
        for cn in fu.scene_groupby_columns: mask_series &= (frvrs_logs_df[cn] == eval(cn))
        frvrs_logs_df.loc[mask_series, new_column_name] = fu.get_is_scene_aborted(scene_df)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 111)
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

,record_count
is_scene_aborted,
False,829116


In [11]:

# Check if all the patient IDs in any run are some variant of Mike and designate those runs as "Orientation"
new_column_name = 'scene_type'
if (new_column_name not in frvrs_logs_df.columns): frvrs_logs_df[new_column_name] = 'Triage'
column_value = 'Orientation'
if (column_value not in frvrs_logs_df.scene_type.unique()):
    
    # Filter out those files from the dataset and mark them
    base_mask_series = frvrs_logs_df.groupby(fu.scene_groupby_columns).patient_id.transform(lambda srs: all(srs.str.lower().str.contains('mike')))
    frvrs_logs_df.loc[base_mask_series, new_column_name] = column_value
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 112)
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

,record_count
scene_type,
Orientation,135937
Triage,693179


In [12]:

# Get a sample with a clear count of responders
new_column_name = 'is_a_one_triage_file'
if (new_column_name in frvrs_logs_df.columns): frvrs_logs_df = frvrs_logs_df.drop(columns=new_column_name)
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df[new_column_name] = False
    for file_name in frvrs_logs_df.file_name.unique():
        
        # Filter out the triage files in this file name
        base_mask_series = (frvrs_logs_df.file_name == file_name)
        mask_series = base_mask_series & (frvrs_logs_df.scene_type == 'Triage') & (frvrs_logs_df.is_scene_aborted == False)
        
        # Get whether the file has only one triage run
        triage_scene_count = len(frvrs_logs_df[mask_series].groupby('scene_id').groups)
        is_a_one_triage_file = bool(triage_scene_count == 1)
        
        frvrs_logs_df.loc[base_mask_series, new_column_name] = is_a_one_triage_file
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 113)
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
(829116, 114)


,record_count
is_a_one_triage_file,
False,462331
True,366785


In [13]:

# Mask voice capture PII. OSU screened all of the **VOICE_COMMAND** and **VOICE_CAPTURE** lines and
# replaced any names with either Max or Jane, regardless of whether the name was that of the responder.
# But, just to make sure...
columns_list = ['voice_command_command_description', 'voice_capture_message']
if not frvrs_logs_df[columns_list].applymap(lambda x: '[PERSON]' in str(x), na_action='ignore').sum().sum():
    import spacy
    try: nlp = spacy.load('en_core_web_sm')
    except OSError as e:
        print(str(e).strip())
        command_str = f'{sys.executable} -m spacy download en_core_web_sm --quiet'
        print(command_str)
        !{command_str}
        nlp = spacy.load('en_core_web_sm')
    import en_core_web_sm
    nlp = en_core_web_sm.load()
    
    mask_series = frvrs_logs_df.voice_command_command_description.isnull() & frvrs_logs_df.voice_capture_message.isnull()
    df = frvrs_logs_df[~mask_series]
    def mask_pii(srs):
        for idx in columns_list:
            new_text = srs[idx]
            if pd.notnull(new_text):
                doc = nlp(new_text)
                for entity in doc.ents:
                    if entity.label_ == 'PERSON': new_text = re.sub('\\b' + entity.text + '\\b', '[PERSON]', new_text)
                srs[idx] = new_text
    
        return srs
    
    for row_index, row_series in df.apply(mask_pii, axis='columns')[columns_list].iterrows():
        for column_name, column_value in row_series.items():
            if pd.notnull(column_value): frvrs_logs_df.loc[row_index, column_name] = column_value
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 113)

In [14]:

# Add a voice capture sentiment score
new_column_name = 'voice_capture_sentiment_score'
if (new_column_name not in frvrs_logs_df.columns):
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    try: sid = SentimentIntensityAnalyzer()
    except LookupError as e:
        print(str(e).strip())
        import nltk
        nltk.download('vader_lexicon')
        sid = SentimentIntensityAnalyzer()
    mask_series = frvrs_logs_df.voice_capture_message.isnull()
    for row_index, row_series in frvrs_logs_df[~mask_series].iterrows():
        voice_capture_message = '\n' + row_series.voice_capture_message
        frvrs_logs_df.loc[row_index, new_column_name] = sid.polarity_scores(voice_capture_message)['compound']
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 114)
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

,record_count
voice_capture_sentiment_score,
-0.9313,1
-0.9022,2
-0.9001,1
-0.8664,1
-0.8658,1
...,...
0.9689,1
0.9704,1
0.9753,1


In [15]:

nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)
print(frvrs_logs_df.shape)

Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv
(829116, 114)
